# Wrangle and Analyze Data

In [39]:
# Import libraries
import pandas as pd
import numpy
import requests
import tweepy 
import json
import config

In [46]:
### Import data ###

# Save csv file as df
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

# Image prediction file to df
image_tsv = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
open('image_predictions.tsv', 'wb').write(image_tsv.content) # write to file
image_pred_df = pd.read_csv('image_predictions.tsv', sep = '\t')

# Connect to tweepy auth and instantiate API
auth = tweepy.OAuthHandler(config.CONSUMER_KEY, config.CONSUMER_SECRET)
auth.set_access_token(config.ACCESS_TOKEN, config.ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

In [5]:
twitter_archive.head() # view archive header

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [117]:
# Create a list to store tweet records
raw_data = []
not_found = [] # list to store IDs of tweets that could not get status on

for i in twitter_archive.tweet_id:
    temp_dict = {} # temporary dict
    try:
        status = api.get_status(i)
        temp_dict['tweet_id'] = i
        temp_dict['retweet_count'] = status._json['retweet_count']
        temp_dict['favorite_count'] = status._json['favorite_count']
        raw_data.append(temp_dict)
    except:
        not_found.append(i)

In [118]:
# write dict to json file
with open('tweet_json.txt', 'w') as json_file:
    json.dump(raw_data, json_file)


In [120]:
# json data to pandas df
tweets_df = pd.read_json('tweet_json.txt')
tweets_df.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7416,35161
1,892177421306343426,5517,30426
2,891815181378084864,3641,22880
3,891689557279858688,7592,38449
4,891327558926688256,8166,36696
